In [41]:
from sympy import symbols, Matrix, sqrt, zeros, pprint

def get_Dr_corrected(m_cutoff):
    """
    Constructs the Drive operator D_R for a coupled Transmon-Cavity system.
    Includes the -h.c. (Hermitian Conjugate) term.
    """
    Omega0, Delta_bd, chi, K = symbols('\Omega_0 \Delta_{bd} chi K', real=True)
    
    atom_levels = 3  # g, e, f
    dim_total = atom_levels * m_cutoff
    
    # 1. Create the 'Raising' part first (as shown in the image)
    Dr_raising = zeros(dim_total, dim_total)

    # Helper for indexing
    def get_idx(level, m):
        return level * m_cutoff + m

    for m in range(m_cutoff):
        idx_g = get_idx(0, m)
        idx_e = get_idx(1, m)
        idx_f = get_idx(2, m)

        # Term 1: Coefficient * |e,m><g,m|
        c1 = Omega0 / (Delta_bd + m * chi)
        Dr_raising[idx_e, idx_g] = c1

        # Term 2: Coefficient * |f,m><e,m|
        c2 = (sqrt(2) * Omega0) / (Delta_bd + m * chi + K)
        Dr_raising[idx_f, idx_e] = c2
        
    # 2. Construct full Dr = (Raising Part) - (Hermitian Conjugate)
    # Since variables are real, Conjugate Transpose (H) is just Transpose (T)
    Dr_full = Dr_raising - Dr_raising.T
    
    return Dr_full

def get_b(m_cutoff):
    """
    Constructs the Lowering operator b.
    """
    atom_levels = 3 
    dim_total = atom_levels * m_cutoff
    b = zeros(dim_total, dim_total)

    def get_idx(level, m):
        return level * m_cutoff + m

    for m in range(m_cutoff):
        idx_g = get_idx(0, m)
        idx_e = get_idx(1, m)
        idx_f = get_idx(2, m)

        # Term 1: |g,m><e,m|
        b[idx_g, idx_e] = 1

        # Term 2: sqrt(2) * |e,m><f,m|
        b[idx_e, idx_f] = sqrt(2)
        
    return b

def get_c(m_cutoff):
    """
    Constructs the operator I_atom (tensor) c_cavity.
    Represents photon annihilation independent of the atomic state.
    """
    atom_levels = 3  # g, e, f
    dim_total = atom_levels * m_cutoff
    op_c = zeros(dim_total, dim_total)

    # Helper for indexing: |level, m> -> flat index
    def get_idx(level, m):
        return level * m_cutoff + m

    # Loop over each atomic level (acting as Identity on the atom)
    for level in range(atom_levels):
        # Loop over photon states m. 
        # c destroys a photon: c|m> = sqrt(m)|m-1>
        # We start from m=1 because m=0 cannot be lowered.
        for m in range(1, m_cutoff):
            
            # Source state: |level, m>
            col_idx = get_idx(level, m)
            
            # Target state: |level, m-1>
            row_idx = get_idx(level, m - 1)
            
            # Matrix element: <m-1| c |m> = sqrt(m)
            op_c[row_idx, col_idx] = sqrt(m)
            
    return op_c

# --- Verification ---



In [62]:
from sympy import symbols, Symbol, Matrix, pprint, latex, Add

def classify_and_group_by_m(matrix, m_cutoff):
    """
    Translates a matrix into symbolic Bra-Ket notation.
    Groups terms first by Delta m (photon jump), and then by specific m subspace.
    
    Returns:
        dict: { delta_m : { m_subspace : symbolic_expression } }
    """
    rows, cols = matrix.shape
    
    # Structure: groups[delta_m][m_index] = expression
    groups = {}

    atom_labels = {0: 'g', 1: 'e', 2: 'f'}

    def get_state(idx):
        level_idx = idx // m_cutoff
        m_val = idx % m_cutoff
        return level_idx, m_val, f"{atom_labels[level_idx]},{m_val}"

    for r in range(rows):
        for c in range(cols):
            coeff = matrix[r, c]
            
            if coeff != 0:
                # 1. Identify indices
                lvl_r, m_r, label_ket = get_state(r)
                lvl_c, m_c, label_bra = get_state(c)
                
                # 2. Determine Groups
                delta_m = abs(m_r - m_c)
                
                # For the subspace key, we use the minimum m involved.
                # e.g., |g,1><e,1| -> m=1
                # e.g., |g,1><e,2| -> m=1 (connects subspace 1 and 2)
                m_key = min(m_r, m_c)

                # 3. Build Symbol
                op_symbol = Symbol(
                    rf"\left|{label_ket}\right\rangle \left\langle {label_bra}\right|"
                )
                term = coeff * op_symbol
                
                # 4. Store in Dictionary
                if delta_m not in groups:
                    groups[delta_m] = {}
                
                if m_key not in groups[delta_m]:
                    groups[delta_m][m_key] = 0
                    
                groups[delta_m][m_key] += term

    return groups


def print_expression(grouped_data):
    # Iterate over photon jumps (Delta m = 0 is the most important for shifts)
    for delta_m in sorted(grouped_data.keys()):
        print(f"========================================")
        print(f" TERMS WITH PHOTON JUMP Δm = {delta_m}")
        print(f"========================================")
        
        subspaces = grouped_data[delta_m]
        
        # Iterate over each m subspace (0, 1, 2...)
        for m_val in sorted(subspaces.keys()):
            print(f"\n  --- Subspace m = {m_val} ---")
            expr = subspaces[m_val]
            print(expr)
            
# ==========================================
# DEMONSTRATION
# ==========================================

# ==========================================
# Demonstration
# ==========================================
m_size = 4 # Check small size
Dr = get_Dr_corrected(m_size)
b = get_b(m_size)
c = get_c(m_size)
Dr_c = Dr * c - c * Dr
Dr_b = Dr * b - b * Dr

bd = b.conjugate().T
bdb = bd * b
Dr_bdb = Dr * bdb - bdb * Dr

Dr_b2 = Dr * Dr_b - Dr_b * Dr
Dr_bdb2 = Dr * Dr_bdb - Dr_bdb * Dr
Dr_c2 = Dr * Dr_c - Dr_c * Dr


In [63]:
grouped_data = classify_and_group_by_m(Dr_b, m_size)
print_expression(grouped_data)

 TERMS WITH PHOTON JUMP Δm = 0

  --- Subspace m = 0 ---
2*\Omega_0*\left|f,0\right\rangle \left\langle f,0\right|/(K + \Delta_{bd}) + \left|e,0\right\rangle \left\langle e,0\right|*(-2*\Omega_0/(K + \Delta_{bd}) + \Omega_0/\Delta_{bd}) + \left|g,0\right\rangle \left\langle f,0\right|*(sqrt(2)*\Omega_0/(K + \Delta_{bd}) - sqrt(2)*\Omega_0/\Delta_{bd}) - \Omega_0*\left|g,0\right\rangle \left\langle g,0\right|/\Delta_{bd}

  --- Subspace m = 1 ---
2*\Omega_0*\left|f,1\right\rangle \left\langle f,1\right|/(K + \Delta_{bd} + chi) - \Omega_0*\left|g,1\right\rangle \left\langle g,1\right|/(\Delta_{bd} + chi) + \left|e,1\right\rangle \left\langle e,1\right|*(-2*\Omega_0/(K + \Delta_{bd} + chi) + \Omega_0/(\Delta_{bd} + chi)) + \left|g,1\right\rangle \left\langle f,1\right|*(sqrt(2)*\Omega_0/(K + \Delta_{bd} + chi) - sqrt(2)*\Omega_0/(\Delta_{bd} + chi))

  --- Subspace m = 2 ---
2*\Omega_0*\left|f,2\right\rangle \left\langle f,2\right|/(K + \Delta_{bd} + 2*chi) - \Omega_0*\left|g,2\right\rang

In [64]:
# $$\mathcal{O} = \sum_m \left[ 
# \underbrace{-2\Omega_0 \left( \frac{1}{\Delta_{bd} + m\chi} - \frac{1}{\Delta_{bd} + K + m\chi} \right) |g,m\rangle\langle e,m|}_{\text{Modified g-e coupling}} 
# + 
# \underbrace{\sqrt{2}\Omega_0 \left( \frac{1}{\Delta_{bd} + m\chi} - \frac{4}{\Delta_{bd} + K + m\chi} \right) |e,m\rangle\langle f,m|}_{\text{Modified e-f coupling}} 
# \right]$$

In [65]:
grouped_data = classify_and_group_by_m(Dr_b2, m_size)
print_expression(grouped_data)

 TERMS WITH PHOTON JUMP Δm = 0

  --- Subspace m = 0 ---
\left|e,0\right\rangle \left\langle f,0\right|*(-2*sqrt(2)*\Omega_0**2/(K + \Delta_{bd})**2 + sqrt(2)*\Omega_0*(-2*\Omega_0/(K + \Delta_{bd}) + \Omega_0/\Delta_{bd})/(K + \Delta_{bd}) + \Omega_0*(sqrt(2)*\Omega_0/(K + \Delta_{bd}) - sqrt(2)*\Omega_0/\Delta_{bd})/\Delta_{bd}) + \left|e,0\right\rangle \left\langle g,0\right|*(-\Omega_0*(-2*\Omega_0/(K + \Delta_{bd}) + \Omega_0/\Delta_{bd})/\Delta_{bd} - \Omega_0**2/\Delta_{bd}**2) + \left|f,0\right\rangle \left\langle e,0\right|*(-2*sqrt(2)*\Omega_0**2/(K + \Delta_{bd})**2 + sqrt(2)*\Omega_0*(-2*\Omega_0/(K + \Delta_{bd}) + \Omega_0/\Delta_{bd})/(K + \Delta_{bd})) + \left|g,0\right\rangle \left\langle e,0\right|*(-sqrt(2)*\Omega_0*(sqrt(2)*\Omega_0/(K + \Delta_{bd}) - sqrt(2)*\Omega_0/\Delta_{bd})/(K + \Delta_{bd}) - \Omega_0*(-2*\Omega_0/(K + \Delta_{bd}) + \Omega_0/\Delta_{bd})/\Delta_{bd} - \Omega_0**2/\Delta_{bd}**2)

  --- Subspace m = 1 ---
\left|e,1\right\rangle \left\langle

In [66]:
Dr_b2

Matrix([
[                                                                                                        0,                                                                                                                                       0,                                                                                                                                               0,                                                                                                                                               0, -sqrt(2)*\Omega_0*(sqrt(2)*\Omega_0/(K + \Delta_{bd}) - sqrt(2)*\Omega_0/\Delta_{bd})/(K + \Delta_{bd}) - \Omega_0*(-2*\Omega_0/(K + \Delta_{bd}) + \Omega_0/\Delta_{bd})/\Delta_{bd} - \Omega_0**2/\Delta_{bd}**2,                                                                                                                                                                                                                                                     

In [67]:
grouped_data = classify_and_group_by_m(Dr_bdb, m_size)
print_expression(grouped_data)

 TERMS WITH PHOTON JUMP Δm = 0

  --- Subspace m = 0 ---
-sqrt(2)*\Omega_0*\left|e,0\right\rangle \left\langle f,0\right|/(K + \Delta_{bd}) - sqrt(2)*\Omega_0*\left|f,0\right\rangle \left\langle e,0\right|/(K + \Delta_{bd}) - \Omega_0*\left|e,0\right\rangle \left\langle g,0\right|/\Delta_{bd} - \Omega_0*\left|g,0\right\rangle \left\langle e,0\right|/\Delta_{bd}

  --- Subspace m = 1 ---
-sqrt(2)*\Omega_0*\left|e,1\right\rangle \left\langle f,1\right|/(K + \Delta_{bd} + chi) - \Omega_0*\left|e,1\right\rangle \left\langle g,1\right|/(\Delta_{bd} + chi) - sqrt(2)*\Omega_0*\left|f,1\right\rangle \left\langle e,1\right|/(K + \Delta_{bd} + chi) - \Omega_0*\left|g,1\right\rangle \left\langle e,1\right|/(\Delta_{bd} + chi)

  --- Subspace m = 2 ---
-sqrt(2)*\Omega_0*\left|e,2\right\rangle \left\langle f,2\right|/(K + \Delta_{bd} + 2*chi) - \Omega_0*\left|e,2\right\rangle \left\langle g,2\right|/(\Delta_{bd} + 2*chi) - sqrt(2)*\Omega_0*\left|f,2\right\rangle \left\langle e,2\right|/(K + \Delta_

In [68]:
grouped_data = classify_and_group_by_m(Dr_bdb2, m_size)
print_expression(grouped_data)

 TERMS WITH PHOTON JUMP Δm = 0

  --- Subspace m = 0 ---
-4*\Omega_0**2*\left|f,0\right\rangle \left\langle f,0\right|/(K + \Delta_{bd})**2 + \left|e,0\right\rangle \left\langle e,0\right|*(4*\Omega_0**2/(K + \Delta_{bd})**2 - 2*\Omega_0**2/\Delta_{bd}**2) + 2*\Omega_0**2*\left|g,0\right\rangle \left\langle g,0\right|/\Delta_{bd}**2

  --- Subspace m = 1 ---
-4*\Omega_0**2*\left|f,1\right\rangle \left\langle f,1\right|/(K + \Delta_{bd} + chi)**2 + 2*\Omega_0**2*\left|g,1\right\rangle \left\langle g,1\right|/(\Delta_{bd} + chi)**2 + \left|e,1\right\rangle \left\langle e,1\right|*(4*\Omega_0**2/(K + \Delta_{bd} + chi)**2 - 2*\Omega_0**2/(\Delta_{bd} + chi)**2)

  --- Subspace m = 2 ---
-4*\Omega_0**2*\left|f,2\right\rangle \left\langle f,2\right|/(K + \Delta_{bd} + 2*chi)**2 + 2*\Omega_0**2*\left|g,2\right\rangle \left\langle g,2\right|/(\Delta_{bd} + 2*chi)**2 + \left|e,2\right\rangle \left\langle e,2\right|*(4*\Omega_0**2/(K + \Delta_{bd} + 2*chi)**2 - 2*\Omega_0**2/(\Delta_{bd} + 2*ch

In [69]:
classify_and_group_by_m(Dr_bdb2, m_size)[0][0]

-4*\Omega_0**2*\left|f,0\right\rangle \left\langle f,0\right|/(K + \Delta_{bd})**2 + \left|e,0\right\rangle \left\langle e,0\right|*(4*\Omega_0**2/(K + \Delta_{bd})**2 - 2*\Omega_0**2/\Delta_{bd}**2) + 2*\Omega_0**2*\left|g,0\right\rangle \left\langle g,0\right|/\Delta_{bd}**2

In [71]:
Dr_bdb

Matrix([
[                    0,                             0,                               0,                               0,               -\Omega_0/\Delta_{bd},                                         0,                                           0,                                           0,                                   0,                                         0,                                           0,                                           0],
[                    0,                             0,                               0,                               0,                                   0,             -\Omega_0/(\Delta_{bd} + chi),                                           0,                                           0,                                   0,                                         0,                                           0,                                           0],
[                    0,                             0,             

In [72]:
grouped_data = classify_and_group_by_m(Dr_c, m_size)
print_expression(grouped_data)

 TERMS WITH PHOTON JUMP Δm = 1

  --- Subspace m = 0 ---
\left|e,0\right\rangle \left\langle f,1\right|*(sqrt(2)*\Omega_0/(K + \Delta_{bd} + chi) - sqrt(2)*\Omega_0/(K + \Delta_{bd})) + \left|e,0\right\rangle \left\langle g,1\right|*(-\Omega_0/(\Delta_{bd} + chi) + \Omega_0/\Delta_{bd}) + \left|f,0\right\rangle \left\langle e,1\right|*(-sqrt(2)*\Omega_0/(K + \Delta_{bd} + chi) + sqrt(2)*\Omega_0/(K + \Delta_{bd})) + \left|g,0\right\rangle \left\langle e,1\right|*(\Omega_0/(\Delta_{bd} + chi) - \Omega_0/\Delta_{bd})

  --- Subspace m = 1 ---
\left|e,1\right\rangle \left\langle f,2\right|*(2*\Omega_0/(K + \Delta_{bd} + 2*chi) - 2*\Omega_0/(K + \Delta_{bd} + chi)) + \left|e,1\right\rangle \left\langle g,2\right|*(-sqrt(2)*\Omega_0/(\Delta_{bd} + 2*chi) + sqrt(2)*\Omega_0/(\Delta_{bd} + chi)) + \left|f,1\right\rangle \left\langle e,2\right|*(-2*\Omega_0/(K + \Delta_{bd} + 2*chi) + 2*\Omega_0/(K + \Delta_{bd} + chi)) + \left|g,1\right\rangle \left\langle e,2\right|*(sqrt(2)*\Omega_0/(\Delta

In [73]:
grouped_data = classify_and_group_by_m(Dr_c2, m_size)
print_expression(grouped_data)

 TERMS WITH PHOTON JUMP Δm = 1

  --- Subspace m = 0 ---
\left|e,0\right\rangle \left\langle e,1\right|*(-sqrt(2)*\Omega_0*(sqrt(2)*\Omega_0/(K + \Delta_{bd} + chi) - sqrt(2)*\Omega_0/(K + \Delta_{bd}))/(K + \Delta_{bd} + chi) + \Omega_0*(-\Omega_0/(\Delta_{bd} + chi) + \Omega_0/\Delta_{bd})/(\Delta_{bd} + chi) - sqrt(2)*\Omega_0*(-sqrt(2)*\Omega_0/(K + \Delta_{bd} + chi) + sqrt(2)*\Omega_0/(K + \Delta_{bd}))/(K + \Delta_{bd}) + \Omega_0*(\Omega_0/(\Delta_{bd} + chi) - \Omega_0/\Delta_{bd})/\Delta_{bd}) + \left|f,0\right\rangle \left\langle f,1\right|*(sqrt(2)*\Omega_0*(-sqrt(2)*\Omega_0/(K + \Delta_{bd} + chi) + sqrt(2)*\Omega_0/(K + \Delta_{bd}))/(K + \Delta_{bd} + chi) + sqrt(2)*\Omega_0*(sqrt(2)*\Omega_0/(K + \Delta_{bd} + chi) - sqrt(2)*\Omega_0/(K + \Delta_{bd}))/(K + \Delta_{bd})) + \left|f,0\right\rangle \left\langle g,1\right|*(-\Omega_0*(-sqrt(2)*\Omega_0/(K + \Delta_{bd} + chi) + sqrt(2)*\Omega_0/(K + \Delta_{bd}))/(\Delta_{bd} + chi) + sqrt(2)*\Omega_0*(-\Omega_0/(\Delta_{b